# Inference Model Test

### Import required libraries

In [1]:
import torch
import torchvision
import torch.nn as nn
import cv2
import numpy as np
import pandas as pd
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import torch.nn.functional as F
import time
import pickle
import sklearn
from sklearn.svm import OneClassSVM
import random
import collections
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
print(torch.__version__)
print(torchvision.__version__)
print('The scikit-learn version is {}.'.format(sklearn.__version__))

1.3.0
0.4.0a0+d31eafa
The scikit-learn version is 0.19.1.


### Load models

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.bn1 = nn.BatchNorm2d(12)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.bn2 = nn.BatchNorm2d(24)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(24*53*53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1,24*53*53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
device = torch.device('cuda')

In [5]:
net1 = Net()
state_dict1 = torch.load('best_model1_pruned.pth')
net1.load_state_dict(state_dict1)

<All keys matched successfully>

In [6]:
net2 = Net()
state_dict2 = torch.load('best_model2_pruned.pth')
net2.load_state_dict(state_dict2)

<All keys matched successfully>

In [7]:
net3 = Net()
state_dict3 = torch.load('best_model3_pruned.pth')
net3.load_state_dict(state_dict3)

<All keys matched successfully>

In [8]:
net1 = net1.to(device)
net2 = net2.to(device)
net3 = net3.to(device)

### Retrieve data

In [9]:
# Retrieve normalisation parameters 

norm_param_df = pd.read_csv('TRG_DATASET_NORM_PARAM.csv')

meanR = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "meanR"].item()
meanG = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "meanG"].item()
meanB = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "meanB"].item()

stdR = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "stdR"].item()
stdG = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "stdG"].item()
stdB = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "stdB"].item()


In [10]:
dataset = datasets.ImageFolder(
    'TEST_0_FINAL/',
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((meanR, meanG, meanB), (stdR, stdG, stdB))
    ])
)
testloader = torch.utils.data.DataLoader(
        dataset,
        shuffle = False,
        batch_size=1)

### Run test Dataset through inference model

In [11]:
threshold = 0.97
threshold_TS = 0.73
temp_factor_net1 = 3.661
temp_factor_net2 = 4.289
temp_factor_net3 = 3.913

In [12]:
total_all1 = 0
correct_all1 = 0
total_sure1 = 0
correct_sure1 = 0
total_all2 = 0
correct_all2 = 0
total_sure2 = 0
correct_sure2 = 0
total_all3 = 0
correct_all3 = 0
total_sure3 = 0
correct_sure3 = 0
total_final = 0
correct_final = 0
TP1 = 0
TN1 = 0
FP1 = 0
FN1 = 0
TP2 = 0
TN2 = 0
FP2 = 0
FN2 = 0
TP3 = 0
TN3 = 0
FP3 = 0
FN3 = 0
TP = 0
TN = 0
FP = 0
FN = 0
m = nn.Softmax(dim=1)

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        net1.eval()
        out1 = net1(images)        
        net2.eval()
        out2 = net2(images)        
        net3.eval()
        out3 = net3(images)

        _, predicted1 = torch.max(out1.data, 1)
        _, predicted2 = torch.max(out2.data, 1)
        _, predicted3 = torch.max(out3.data, 1)
  
        predicted_soft1 = m(out1)
        predicted_soft2 = m(out2)
        predicted_soft3 = m(out3)
        true = labels.item() - 1

        predicted_soft1_TS = F.softmax(out1/temp_factor_net1, dim=1)
        predicted_soft2_TS = F.softmax(out2/temp_factor_net2, dim=1)
        predicted_soft3_TS = F.softmax(out3/temp_factor_net3, dim=1)
        
        if np.amax(predicted_soft1.cpu().numpy()) > threshold:
            total_sure1 += labels.size(0)
            correct_sure1 += (predicted1 == true).sum().item()
        total_all1 += labels.size(0)
        correct_all1 += (predicted1 == true).sum().item()
        TP1 += (predicted1 == labels - 1 & labels - 1 == 0).sum().item()
        TN1 += (predicted1 == labels - 1 & labels - 1 == 1).sum().item()
        FP1 += (predicted1 != labels - 1 & labels - 1 == 1).sum().item()
        FN1 += (predicted1 != labels - 1 & labels - 1 == 0).sum().item()
        prediction1 = predicted1.item()
        if predicted1 == true:
            correct1 = 1
        else:
            correct1 = 0

        if np.amax(predicted_soft1.cpu().numpy()) > threshold:
            sure1 = 1
        else:
            sure1 = 0
          
        if np.amax(predicted_soft2.cpu().numpy()) > threshold:
            total_sure2 += labels.size(0)
            correct_sure2 += (predicted2 == true).sum().item()
        total_all2 += labels.size(0)
        correct_all2 += (predicted2 == true).sum().item()
        TP2 += (predicted2 == labels - 1 & labels - 1 == 0).sum().item()
        TN2 += (predicted2 == labels - 1 & labels - 1 == 1).sum().item()
        FP2 += (predicted2 != labels - 1 & labels - 1 == 1).sum().item()
        FN2 += (predicted2 != labels - 1 & labels - 1 == 0).sum().item()
        prediction2 = predicted2.item()
        if predicted2 == true:
            correct2 = 1
        else:
            correct2 = 0

        if np.amax(predicted_soft2.cpu().numpy()) > threshold:
            sure2 = 1
        else:
            sure2 = 0

        if np.amax(predicted_soft3.cpu().numpy()) > threshold:
            total_sure3 += labels.size(0)
            correct_sure3 += (predicted3 == true).sum().item()
        total_all3 += labels.size(0)
        correct_all3 += (predicted3 == true).sum().item()
        TP3 += (predicted3 == labels - 1 & labels - 1 == 0).sum().item()
        TN3 += (predicted3 == labels - 1 & labels - 1 == 1).sum().item()
        FP3 += (predicted3 != labels - 1 & labels - 1 == 1).sum().item()
        FN3 += (predicted3 != labels - 1 & labels - 1 == 0).sum().item()
        prediction3 = predicted3.item()
        if predicted3 == true:
            correct3 = 1
        else:
            correct3 = 0

        if np.amax(predicted_soft3.cpu().numpy()) > threshold:
            sure3 = 1
        else:
            sure3 = 0


        if sure1 + sure2 + sure3 == 3:
            if prediction1 + prediction2 + prediction3 >=2:
                final_prediction = 1
            else:
                final_prediction = 0
        elif sure1 + sure2 + sure3 == 2:
            if sure1 == 0:
                if prediction2 + prediction3 == 2:
                    final_prediction = 1
                elif prediction2 + prediction3 == 0:
                    final_prediction = 0
                else:
                    final_prediction = -1
            if sure2 == 0:
                if prediction1 + prediction3 ==2:
                    final_prediction = 1
                elif prediction1 + prediction3 ==0:
                    final_prediction = 0
                else:
                    final_prediction = -1
            if sure3 == 0:
                if prediction2 + prediction1 ==2:
                    final_prediction = 1
                elif prediction2 + prediction1 ==0:
                    final_prediction = 0
                else:
                    final_prediction = -1
            else:
                final_prediction = -1
          
        if final_prediction != -1:
            total_final += labels.size(0)
            correct_final += (final_prediction == labels - 1).sum().item()
            TP += (final_prediction == labels - 1 & labels - 1 == 0).sum().item()
            TN += (final_prediction == labels - 1 & labels - 1 == 1).sum().item()
            FP += (final_prediction != labels - 1 & labels - 1 == 1).sum().item()
            FN += (final_prediction != labels - 1 & labels - 1 == 0).sum().item()
                  

test_accuracy_all1 = 100 * correct_all1 / total_all1
test_accuracy_sure1 = 100 * correct_sure1 / total_sure1
test_accuracy_all2 = 100 * correct_all2 / total_all2
test_accuracy_sure2 = 100 * correct_sure2 / total_sure2
test_accuracy_all3 = 100 * correct_all3 / total_all3
test_accuracy_sure3 = 100 * correct_sure3 / total_sure3
test_accuracy_final = 100 * correct_final / total_final
print("Threshold:", threshold)
print("Model1:", test_accuracy_all1, test_accuracy_sure1, 100 * total_sure1/total_all1, TP1, TN1, FP1, FN1)
print("Model2:", test_accuracy_all2, test_accuracy_sure2, 100 * total_sure2/total_all2, TP2, TN2, FP2, FN2)
print("Model3:", test_accuracy_all3, test_accuracy_sure3, 100 * total_sure3/total_all3, TP3, TN3, FP3, FN3)
print("Final: ", test_accuracy_final, '                 ', 100 * total_final/total_all3, TP, TN, FP, FN)

Threshold: 0.97
Model1: 79.55493741307372 85.46712802768167 80.3894297635605 277 295 70 77
Model2: 77.88595271210013 88.3629191321499 70.51460361613351 280 280 85 74
Model3: 77.32962447844228 85.8508604206501 72.73991655076495 236 320 45 118
Final:  91.57894736842105                   79.27677329624478 253 269 20 28
